IMPORT, READ_CSV

In [2]:
#Standard imports
import pandas as pd
import string
import re
import numpy as np

#SKlearn
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

#NLTK
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer, sent_tokenize
from nltk import WordNetLemmatizer # lemmatizer using WordNet
from nltk.corpus import wordnet # imports WordNet
from nltk import pos_tag # nltk's native part of speech tagging

[nltk_data] Downloading package stopwords to /Users/javm/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/javm/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /Users/javm/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/javm/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
# import tensorflow as tf
# tf.config.set_visible_devices([], 'GPU')
# tf.debugging.set_log_device_placement(True)

# # Create some tensors and perform an operation
# a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
# b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
# c = tf.matmul(a, b)

# print(c)

In [4]:
df = pd.read_csv('/Users/javm/Desktop/Projects/Book-Recommendations/books_with_blurbs.csv')

In [5]:
df

,ISBN,Title,Author,Year,Publisher,Blurb
0,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,"Here, for the first time in paperback, is an o..."
1,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,"The fascinating, true story of the world's dea..."
2,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,Winnie and Helen have kept each others worst s...
3,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000,Berkley Publishing Group,Historians and inquisitive laymen alike love t...
4,1881320189,Goodbye to the Buttermilk Sky,Julia Oliver,1994,River City Pub,This highly praised first novel by fiction wri...
...,...,...,...,...,...,...
57505,0451458877,Tainted Trail,Wen Spencer,2002,Roc,"Ukiah Oregon, half-man and half-alien raised b..."
57506,0399148736,Twelve Mile Limit,Randy Wayne White,2002,Penguin Putnam,"On a Friday in early November, four people hea..."
57507,0399148841,The Man With the Red Tattoo (James Bond 007),Raymond Benson,2002,Putnam Publishing Group,On a quiet late-night flight from Tokyo to Lon...
57508,0553578979,"Iron Fist (Star Wars: X-Wing Series, Book 6)",Aaron Allston,1998,Bantam,They are the Rebel Alliance's ultimate strike ...


TEXT COMPILATION

In [6]:
df['Whole_Text']= df['Title'] + " " + df ['Author'] + " " + df['Publisher'] + " " + df['Blurb']

In [7]:
train_df = df.drop(columns = ['ISBN'])

In [8]:
train_df

,Title,Author,Year,Publisher,Blurb,Whole_Text
0,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,"Here, for the first time in paperback, is an o...",Decision in Normandy Carlo D'Este HarperPerenn...
1,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,"The fascinating, true story of the world's dea...",Flu: The Story of the Great Influenza Pandemic...
2,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,Winnie and Helen have kept each others worst s...,The Kitchen God's Wife Amy Tan Putnam Pub Grou...
3,What If?: The World's Foremost Military Histor...,Robert Cowley,2000,Berkley Publishing Group,Historians and inquisitive laymen alike love t...,What If?: The World's Foremost Military Histor...
4,Goodbye to the Buttermilk Sky,Julia Oliver,1994,River City Pub,This highly praised first novel by fiction wri...,Goodbye to the Buttermilk Sky Julia Oliver Riv...
...,...,...,...,...,...,...
57505,Tainted Trail,Wen Spencer,2002,Roc,"Ukiah Oregon, half-man and half-alien raised b...","Tainted Trail Wen Spencer Roc Ukiah Oregon, ha..."
57506,Twelve Mile Limit,Randy Wayne White,2002,Penguin Putnam,"On a Friday in early November, four people hea...",Twelve Mile Limit Randy Wayne White Penguin Pu...
57507,The Man With the Red Tattoo (James Bond 007),Raymond Benson,2002,Putnam Publishing Group,On a quiet late-night flight from Tokyo to Lon...,The Man With the Red Tattoo (James Bond 007) R...
57508,"Iron Fist (Star Wars: X-Wing Series, Book 6)",Aaron Allston,1998,Bantam,They are the Rebel Alliance's ultimate strike ...,"Iron Fist (Star Wars: X-Wing Series, Book 6) A..."


In [9]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57510 entries, 0 to 57509
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Title       57510 non-null  object
 1   Author      57510 non-null  object
 2   Year        57510 non-null  int64 
 3   Publisher   57510 non-null  object
 4   Blurb       57510 non-null  object
 5   Whole_Text  57510 non-null  object
dtypes: int64(1), object(5)
memory usage: 2.6+ MB


PREPROCESSING

In [10]:
class TextPreprocessor(BaseEstimator, TransformerMixin):
    
    def __init__(self):
    
        pass
    
    def fit(self, data, y = 0):
        
        return self
    
    def transform(self, data, y = 0):
        fully_normalized_corpus = data.apply(self.process_doc)
        
        return fully_normalized_corpus
        
    def process_doc(self, doc):

        wnl = WordNetLemmatizer()
        stop_words = stopwords.words('english')
        
        def pos_tagger(nltk_tag):
            if nltk_tag.startswith('J'):
                return wordnet.ADJ
            elif nltk_tag.startswith('V'):
                return wordnet.VERB
            elif nltk_tag.startswith('N'):
                return wordnet.NOUN
            elif nltk_tag.startswith('R'):
                return wordnet.ADV
            else:         
                return None

        doc_norm = [tok.lower() for tok in word_tokenize(doc) if (tok.isalpha())]

        wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tag(doc_norm))) 
        doc_norm = [wnl.lemmatize(token, pos) for token, pos in wordnet_tagged if pos is not None]

        return " ".join(doc_norm)


In [11]:
tp = TextPreprocessor()

In [12]:
train_df['norm_text'] = tp.fit_transform(train_df['Whole_Text'])

In [13]:
train_df = train_df.drop(columns = ['Whole_Text'])

In [14]:
text = train_df['norm_text']

In [15]:
# def clean_text(corpus):
#     cleaned_text = []
#     cleaned_text2 = []
#     corpus = re.sub("'", "", corpus)
#     corpus = re.sub("(\\d|\\W)+"," ",corpus)
#     corpus = re.sub("/[^A-Z0-9]+/ig", "",corpus)
#     corpus = corpus.replace('_','')
#     cleaned_text = [wn.lemmatize(word, pos = 'v') for word in word_tokenize(corpus.lower()) if black_txt(word)]
#     cleaned_text2 = [word for word in cleaned_text if black_txt(word)]
#     return " ".join(cleaned_text2)

In [16]:
vectorizer = TfidfVectorizer(min_df = 3, max_df = 10, max_features = 5000)
vectors = vectorizer.fit_transform(text)

In [17]:
vec_df = pd.DataFrame(vectors.toarray(),
                     columns=[k for k, v in sorted(vectorizer.vocabulary_.items(), 
                     key=lambda item: item[1])])

In [18]:
vec_df

,aahz,abandona,abba,abingdon,abnormally,abort,abrahams,abram,absalom,absolution,...,την,της,τον,του,жизни,на,не,به,في,من
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57505,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
57506,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
57507,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
57508,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [91]:
feature_names = vectorizer.get_feature_names_out()
dense = vectors.todense()
denselist = dense.tolist()

: 

: 

In [19]:
book_search = input('What are you looking for: ')

In [20]:
print(book_search)

i want to read a book about dragons and thier history


In [21]:
search = tp.fit_transform(pd.Series(book_search))

In [22]:
search

0    i want read book dragon thier history
dtype: object

In [23]:

search_vec = vectorizer.transform(search)

In [25]:
from sklearn.metrics.pairwise import cosine_similarity

In [27]:
test = cosine_similarity(search_vec, vectors)
#vec_df.apply(lambda x: cosine_similarity(x,search_vec))


In [28]:
test

array([[0., 0., 0., ..., 0., 0., 0.]])

In [35]:
best_book = np.argmax(test[0])

print('Title : {}'.format(train_df.loc[best_book, 'Title']))
print('Author : {}'.format(train_df.loc[best_book, 'Author']))
print('Publisher : {}'.format(train_df.loc[best_book, 'Publisher']))
print('Blurb : {}'.format(train_df.loc[best_book, 'Blurb']))

Title : Decision in Normandy
Author : Carlo D'Este
Publisher : HarperPerennial
Blurb : Here, for the first time in paperback, is an outstanding military history that offers a dramatic new perspective on the Allied campaign that began with the invasion of the D-Day beaches of Normandy. Nationa advertising in Military History.
